In [1]:
import pandas as pd
import numpy as np
from time import time

In [2]:
df_product = pd.read_csv('data/20_production.csv')
df_nj_weather = pd.read_csv('data/21_nj_weather.csv')
df_pa_weather = pd.read_csv('data/22_pa_weather.csv')

In [3]:
df_nj_weather['weather_time'] = df_nj_weather['dt'].apply(lambda x: pd.Timestamp(x, unit='s', tz='America/New_York'))
df_pa_weather['weather_time'] = df_pa_weather['dt'].apply(lambda x: pd.Timestamp(x, unit='s', tz='America/New_York'))
df_nj_weather

,dt,dt_iso,timezone,city_name,lat,lon,temp,feels_like,temp_min,temp_max,...,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon,weather_time
0,1451606400,2016-01-01 00:00:00 +0000 UTC,-18000,Middlesex,40.572603,-74.492654,43.93,37.06,42.80,46.30,...,NaN,NaN,NaN,NaN,75,803,Clouds,broken clouds,04n,2015-12-31 19:00:00-05:00
1,1451610000,2016-01-01 01:00:00 +0000 UTC,-18000,Middlesex,40.572603,-74.492654,43.65,38.10,42.80,46.02,...,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n,2015-12-31 20:00:00-05:00
2,1451613600,2016-01-01 02:00:00 +0000 UTC,-18000,Middlesex,40.572603,-74.492654,43.27,36.12,42.80,45.11,...,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n,2015-12-31 21:00:00-05:00
3,1451617200,2016-01-01 03:00:00 +0000 UTC,-18000,Middlesex,40.572603,-74.492654,42.62,36.14,42.08,44.20,...,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n,2015-12-31 22:00:00-05:00
4,1451620800,2016-01-01 04:00:00 +0000 UTC,-18000,Middlesex,40.572603,-74.492654,42.08,35.42,41.00,43.56,...,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n,2015-12-31 23:00:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46641,1609441200,2020-12-31 19:00:00 +0000 UTC,-18000,Middlesex,40.572603,-74.492654,43.36,34.21,41.00,45.00,...,NaN,NaN,NaN,NaN,75,803,Clouds,broken clouds,04d,2020-12-31 14:00:00-05:00
46642,1609444800,2020-12-31 20:00:00 +0000 UTC,-18000,Middlesex,40.572603,-74.492654,42.30,33.82,39.99,44.01,...,0.76,NaN,NaN,NaN,90,500,Rain,light rain,10d,2020-12-31 15:00:00-05:00
46643,1609448400,2020-12-31 21:00:00 +0000 UTC,-18000,Middlesex,40.572603,-74.492654,41.18,34.99,39.00,42.80,...,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04d,2020-12-31 16:00:00-05:00
46644,1609452000,2020-12-31 22:00:00 +0000 UTC,-18000,Middlesex,40.572603,-74.492654,40.30,35.24,37.99,42.07,...,NaN,NaN,NaN,NaN,75,803,Clouds,broken clouds,04n,2020-12-31 17:00:00-05:00


In [4]:
def tz_y_m_d(ts):

    year = ts.year
    quarter = ts.quarter
    month = ts.month
    day = ts.day
    hour = ts.hour

    return year, quarter, month, day, hour

In [5]:
def round_to_hour(timestamp):

    # due to excel convert to csv, all format are lost and original timestamp has transformed to 'string'
    # so we add a duplicate process to convert 'string' to 'timestamp'
    timestamp = pd.Timestamp(timestamp)
    
    # because of summer time and winter time rules, some UTC date can't conver to EST
    try:
        date_hour = timestamp.round(freq='H').tz_localize(tz='America/New_York', nonexistent='shift_forward')
    except:
        print('This data cant convert correctly {}'.format(timestamp.round(freq='H')))
        date_hour = timestamp.round(freq='H').tz_localize(tz='America/New_York',
                                                          ambiguous='NaT',
                                                          nonexistent='shift_forward')
        print('This date replaced by {}'.format('NaT'))
    return date_hour

In [6]:
df_product['start_time'] = df_product['StartDate'].apply(round_to_hour)

This data cant convert correctly 2016-11-06 01:00:00
This date replaced by NaT
This data cant convert correctly 2017-11-05 01:00:00
This date replaced by NaT


In [7]:
df_nj_weather.head(5)

,dt,dt_iso,timezone,city_name,lat,lon,temp,feels_like,temp_min,temp_max,...,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon,weather_time
0,1451606400,2016-01-01 00:00:00 +0000 UTC,-18000,Middlesex,40.572603,-74.492654,43.93,37.06,42.80,46.30,...,NaN,NaN,NaN,NaN,75,803,Clouds,broken clouds,04n,2015-12-31 19:00:00-05:00
1,1451610000,2016-01-01 01:00:00 +0000 UTC,-18000,Middlesex,40.572603,-74.492654,43.65,38.10,42.80,46.02,...,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n,2015-12-31 20:00:00-05:00
2,1451613600,2016-01-01 02:00:00 +0000 UTC,-18000,Middlesex,40.572603,-74.492654,43.27,36.12,42.80,45.11,...,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n,2015-12-31 21:00:00-05:00
3,1451617200,2016-01-01 03:00:00 +0000 UTC,-18000,Middlesex,40.572603,-74.492654,42.62,36.14,42.08,44.20,...,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n,2015-12-31 22:00:00-05:00
4,1451620800,2016-01-01 04:00:00 +0000 UTC,-18000,Middlesex,40.572603,-74.492654,42.08,35.42,41.00,43.56,...,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n,2015-12-31 23:00:00-05:00


In [8]:
from datetime import timedelta

df_product['end_time']=df_product.apply(lambda x: x.start_time+timedelta(hours=np.ceil(x.DryingTime_Hrs)), axis=1)

In [9]:
df_product.head(5)

,StartDate,BatchNumber,CustItem,Dryer,Flow,Hygroscopicity,Bulk Density,Moisture,ScheduledDryQty,ActualDryQty,YieldPercentage,DryingTime_Hrs,CleanTime_Hrs,DownTime_Hrs,TotalRunTime_Hrs,ProdLine,Rate,start_time,end_time
0,2016-01-02 08:54:58.000,NJSD313825,10013-0000,Dryer 04,Good,Medium,Target = 0.30 g/mL,2 to 4% Max,333.7357,352.2000,105.5,2.4,5.1,0.0,7.5,Fragrance,146.750000,2016-01-02 09:00:00-05:00,2016-01-02 12:00:00-05:00
1,2016-01-02 11:06:57.863,NJSD312807,10558-0000,Dryer 03,Good,Medium,Record,10% Max.,4270.4120,3935.3999,92.2,8.4,0.2,4.2,12.8,Fragrance,468.499988,2016-01-02 11:00:00-05:00,2016-01-02 20:00:00-05:00
2,2016-01-02 21:40:51.470,NJSD313826,10447-0000,Dryer 04,Good,Medium,NaN,3% Max,1353.0969,1195.9000,88.4,9.7,5.7,0.9,16.2,Fragrance,123.288660,2016-01-02 22:00:00-05:00,2016-01-03 08:00:00-05:00
3,2016-01-02 22:45:11.673,NJSD312677,24164-0000,Dryer 02,Good,Medium,NaN,5% Max,19549.0975,18185.1000,93.0,54.2,11.1,7.6,72.8,Flavors,335.518450,2016-01-02 23:00:00-05:00,2016-01-05 06:00:00-05:00
4,2016-01-03 00:10:32.910,NJSD314508,10558-0000,Dryer 03,Good,Medium,Record,10% Max.,4282.1720,3373.1999,78.8,8.2,0.1,3.5,11.8,Fragrance,411.365841,2016-01-03 00:00:00-05:00,2016-01-03 09:00:00-05:00


In [10]:
df_nj_product = df_product[df_product['BatchNumber'].str.contains('NJ', regex=False)]
df_pa_product = df_product[df_product['BatchNumber'].str.contains('PA', regex=False)]

In [11]:
from pandasql import sqldf

q1="""
    select *
    from df_nj_product as pd
    left join df_nj_weather as we 
    on we.weather_time > pd.start_time
    and we.weather_time < pd.end_time"""
df_nj_merged = sqldf(q1)

In [12]:
df_nj_merged.head(10)

,StartDate,BatchNumber,CustItem,Dryer,Flow,Hygroscopicity,Bulk Density,Moisture,ScheduledDryQty,ActualDryQty,...,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon,weather_time
0,2016-01-02 08:54:58.000,NJSD313825,10013-0000,Dryer 04,Good,Medium,Target = 0.30 g/mL,2 to 4% Max,333.7357,352.2000,...,NaN,NaN,NaN,NaN,1.0,800.0,Clear,sky is clear,01d,2016-01-02 10:00:00.000000
1,2016-01-02 08:54:58.000,NJSD313825,10013-0000,Dryer 04,Good,Medium,Target = 0.30 g/mL,2 to 4% Max,333.7357,352.2000,...,NaN,NaN,NaN,NaN,75.0,803.0,Clouds,broken clouds,04d,2016-01-02 11:00:00.000000
2,2016-01-02 11:06:57.863,NJSD312807,10558-0000,Dryer 03,Good,Medium,Record,10% Max.,4270.4120,3935.3999,...,NaN,NaN,NaN,NaN,90.0,804.0,Clouds,overcast clouds,04d,2016-01-02 12:00:00.000000
3,2016-01-02 11:06:57.863,NJSD312807,10558-0000,Dryer 03,Good,Medium,Record,10% Max.,4270.4120,3935.3999,...,NaN,NaN,NaN,NaN,20.0,801.0,Clouds,few clouds,02d,2016-01-02 13:00:00.000000
4,2016-01-02 11:06:57.863,NJSD312807,10558-0000,Dryer 03,Good,Medium,Record,10% Max.,4270.4120,3935.3999,...,NaN,NaN,NaN,NaN,1.0,800.0,Clear,sky is clear,01d,2016-01-02 14:00:00.000000
5,2016-01-02 11:06:57.863,NJSD312807,10558-0000,Dryer 03,Good,Medium,Record,10% Max.,4270.4120,3935.3999,...,NaN,NaN,NaN,NaN,20.0,801.0,Clouds,few clouds,02d,2016-01-02 15:00:00.000000
6,2016-01-02 11:06:57.863,NJSD312807,10558-0000,Dryer 03,Good,Medium,Record,10% Max.,4270.4120,3935.3999,...,NaN,NaN,NaN,NaN,90.0,804.0,Clouds,overcast clouds,04d,2016-01-02 16:00:00.000000
7,2016-01-02 11:06:57.863,NJSD312807,10558-0000,Dryer 03,Good,Medium,Record,10% Max.,4270.4120,3935.3999,...,NaN,NaN,NaN,NaN,40.0,802.0,Clouds,scattered clouds,03n,2016-01-02 17:00:00.000000
8,2016-01-02 11:06:57.863,NJSD312807,10558-0000,Dryer 03,Good,Medium,Record,10% Max.,4270.4120,3935.3999,...,NaN,NaN,NaN,NaN,1.0,800.0,Clear,sky is clear,01n,2016-01-02 18:00:00.000000
9,2016-01-02 11:06:57.863,NJSD312807,10558-0000,Dryer 03,Good,Medium,Record,10% Max.,4270.4120,3935.3999,...,NaN,NaN,NaN,NaN,1.0,800.0,Clear,sky is clear,01n,2016-01-02 19:00:00.000000


In [13]:
q2="""
    select *
    from df_pa_product as pd
    left join df_pa_weather as we 
    on we.weather_time > pd.start_time
    and we.weather_time < pd.end_time"""
df_pa_merged = sqldf(q2)

In [14]:
df_pa_merged.head(5)

,StartDate,BatchNumber,CustItem,Dryer,Flow,Hygroscopicity,Bulk Density,Moisture,ScheduledDryQty,ActualDryQty,...,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon,weather_time
0,2016-01-03 05:09:32.667,PASD313468,10456-0000,Dryer 08,Good,Medium,None,0.028,26941.1765,23100.0,...,NaN,NaN,NaN,NaN,1.0,800.0,Clear,sky is clear,01n,2016-01-03 06:00:00.000000
1,2016-01-03 05:09:32.667,PASD313468,10456-0000,Dryer 08,Good,Medium,None,0.028,26941.1765,23100.0,...,NaN,NaN,NaN,NaN,1.0,800.0,Clear,sky is clear,01n,2016-01-03 07:00:00.000000
2,2016-01-03 05:09:32.667,PASD313468,10456-0000,Dryer 08,Good,Medium,None,0.028,26941.1765,23100.0,...,NaN,NaN,NaN,NaN,1.0,800.0,Clear,sky is clear,01d,2016-01-03 08:00:00.000000
3,2016-01-03 05:09:32.667,PASD313468,10456-0000,Dryer 08,Good,Medium,None,0.028,26941.1765,23100.0,...,NaN,NaN,NaN,NaN,1.0,800.0,Clear,sky is clear,01d,2016-01-03 09:00:00.000000
4,2016-01-03 05:09:32.667,PASD313468,10456-0000,Dryer 08,Good,Medium,None,0.028,26941.1765,23100.0,...,NaN,NaN,NaN,NaN,1.0,800.0,Clear,sky is clear,01d,2016-01-03 10:00:00.000000


In [15]:
nj = df_nj_merged.groupby('BatchNumber')[['temp','pressure','humidity','wind_speed','wind_deg','clouds_all']].describe(percentiles=[0.25, 0.5, 0.75])

In [16]:
nj.columns = ['_'.join(col).rstrip('_') for col in nj.columns.values]
nj_product_weather = df_nj_product.merge(nj, how='left', on='BatchNumber')
nj_product_weather

,StartDate,BatchNumber,CustItem,Dryer,Flow,Hygroscopicity,Bulk Density,Moisture,ScheduledDryQty,ActualDryQty,...,wind_deg_75%,wind_deg_max,clouds_all_count,clouds_all_mean,clouds_all_std,clouds_all_min,clouds_all_25%,clouds_all_50%,clouds_all_75%,clouds_all_max
0,2016-01-02 08:54:58.000,NJSD313825,10013-0000,Dryer 04,Good,Medium,Target = 0.30 g/mL,2 to 4% Max,333.7357,352.2000,...,287.5,290.0,2.0,38.000000,52.325902,1.0,19.5,38.0,56.5,75.0
1,2016-01-02 11:06:57.863,NJSD312807,10558-0000,Dryer 03,Good,Medium,Record,10% Max.,4270.4120,3935.3999,...,312.5,330.0,8.0,32.875000,37.714861,1.0,1.0,20.0,52.5,90.0
2,2016-01-02 21:40:51.470,NJSD313826,10447-0000,Dryer 04,Good,Medium,NaN,3% Max,1353.0969,1195.9000,...,227.0,290.0,9.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
3,2016-01-02 22:45:11.673,NJSD312677,24164-0000,Dryer 02,Good,Medium,NaN,5% Max,19549.0975,18185.1000,...,340.0,360.0,54.0,9.111111,23.427662,1.0,1.0,1.0,1.0,90.0
4,2016-01-03 00:10:32.910,NJSD314508,10558-0000,Dryer 03,Good,Medium,Record,10% Max.,4282.1720,3373.1999,...,234.5,290.0,8.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6212,2020-12-27 02:02:32.617,NJSD378398,45723-0000,Dryer 01,Good,Medium,NaN,5.0% MAX,1822.5465,1792.0000,...,304.0,304.0,6.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
6213,2020-12-27 14:47:34.140,NJSD378412,10093-0000,Dryer 01,Good,Medium,NaN,5% Max,1683.3335,1600.0000,...,179.5,190.0,4.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
6214,2020-12-27 16:29:14.457,NJSD379264,44097-0000,Dryer 02,Good,Medium,NaN,5.0% MAX,6488.6985,6200.0000,...,189.5,203.0,11.0,20.727273,36.169299,1.0,1.0,1.0,20.5,90.0
6215,2020-12-28 06:35:56.627,NJSD378095,10247-0000,Dryer 03,Good,Medium,NaN,TARGET <3.5% MAX 4.5%,19073.2326,18400.0000,...,309.0,360.0,86.0,39.895349,40.701215,1.0,1.0,20.0,90.0,90.0


In [17]:
pa = df_pa_merged.groupby('BatchNumber')[['temp','pressure','humidity','wind_speed','wind_deg','clouds_all']].describe(percentiles=[0.25, 0.5, 0.75])

In [18]:
pa.columns = ['_'.join(col).rstrip('_') for col in pa.columns.values]
pa_product_weather = df_pa_product.merge(pa, how='left', on='BatchNumber')
pa_product_weather

,StartDate,BatchNumber,CustItem,Dryer,Flow,Hygroscopicity,Bulk Density,Moisture,ScheduledDryQty,ActualDryQty,...,wind_deg_75%,wind_deg_max,clouds_all_count,clouds_all_mean,clouds_all_std,clouds_all_min,clouds_all_25%,clouds_all_50%,clouds_all_75%,clouds_all_max
0,2016-01-03 05:09:32.667,PASD313468,10456-0000,Dryer 08,Good,Medium,NaN,0.028,26941.1765,23100.0000,...,335.0,350.0,43.0,29.883721,39.392536,1.0,1.0,1.0,82.5,90.0
1,2016-01-03 13:58:36.000,PASD313912,40556-0000,Dryer 09,Good,Medium,NaN,5% Max,393.6016,350.0000,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-01-05 03:52:44.097,PASD314541,40738-0000,Dryer 09,Good,Medium,NaN,5% Max,432.9630,388.9000,...,325.0,360.0,27.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
3,2016-01-05 16:26:23.280,PASD314352,10091-0000,Dryer 07,Poor,Medium,Tapped 0.5 - 0.7,5% Max,5400.8864,4407.9999,...,278.0,345.0,55.0,23.818182,38.221620,1.0,1.0,1.0,57.5,90.0
4,2016-01-06 14:30:21.370,PASD313915,23489-0000,Dryer 09,Good,Medium,NaN,5% MAX,575.6400,518.8000,...,275.0,338.0,27.0,31.000000,40.959642,1.0,1.0,1.0,82.5,90.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3748,2020-12-28 00:25:25.440,PASD377548,48061-0000,Dryer 06,Good,Medium,NaN,5.0% MAX,31807.1134,29100.0000,...,300.0,360.0,62.0,35.919355,39.341839,1.0,1.0,1.0,75.0,90.0
3749,2020-12-28 09:04:23.930,PASD372294,48025-0000,Dryer 10,Good,Medium,NaN,MIN= 2.5; Max = 4.0% (Target = 2.5 – 3.0%),21762.1510,19650.0000,...,305.5,360.0,35.0,44.685714,39.565858,1.0,1.0,40.0,82.5,90.0
3750,2020-12-29 12:28:05.080,PASD380104,41902-0000,Dryer 07,Good,Medium,NaN,5% MAX,490.0008,300.0000,...,315.0,320.0,2.0,38.000000,52.325902,1.0,19.5,38.0,56.5,75.0
3751,2020-12-29 21:33:47.500,PASD377869,43041-0000,Dryer 07,Good,Medium,Record once after startup (tapped),Target 2% MAX 3%,21684.3832,20360.0000,...,220.0,360.0,31.0,34.612903,42.052885,1.0,1.0,1.0,90.0,90.0


In [19]:
df_all = pd.concat([nj_product_weather, pa_product_weather])
df_all.to_csv('data/all_raw_data.csv', index=False)